In [ ]:
from flask import Flask, jsonify, request
import threading
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import os

app = Flask(__name__)

# Словарь для хранения состояния выполнения для каждого этапа
progress = {
    'run_rss': 0,
    'run_scrape': 0,
}

# Функция для выполнения Jupyter notebook
def execute_notebook(notebook_path, progress_key):
    progress[progress_key] = 0
    try:
        with open(notebook_path) as f:
            nb = nbformat.read(f, as_version=4)
        ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
        ep.preprocess(nb, {'metadata': {'path': './'}})
        progress[progress_key] = 100
    except Exception as e:
        progress[progress_key] = -1
        print(f"Failed to execute notebook {notebook_path}: {e}")

@app.route('/run_rss', methods=['GET'])
def run_rss():
    token = request.args.get('token')
    if token != 'your_secret_token':
        return jsonify({"error": "Invalid token"}), 403

    notebook_path = 'rss.ipynb'  # Укажите путь к вашему ноутбуку
    if not os.path.exists(notebook_path):
        return jsonify({"error": "Notebook not found"}), 400

    # Запускаем выполнение ноутбука в отдельном потоке, чтобы не блокировать основной поток
    thread = threading.Thread(target=execute_notebook, args=(notebook_path, 'run_rss'))
    thread.start()
    
    return jsonify({
        "status": "RSS notebook execution started",
        "notebook": notebook_path,
        "message": "Please check results: https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit?gid=2038294867#gid=2038294867"
    }), 200

@app.route('/run_scrape', methods=['GET'])
def run_scrape():
    token = request.args.get('token')
    if token != 'your_secret_token':
        return jsonify({"error": "Invalid token"}), 403

    notebook_path = 'scrape.ipynb'  # Укажите путь к вашему ноутбуку
    if not os.path.exists(notebook_path):
        return jsonify({"error": "Notebook not found"}), 400

    # Запускаем выполнение ноутбука в отдельном потоке, чтобы не блокировать основной поток
    thread = threading.Thread(target=execute_notebook, args=(notebook_path, 'run_scrape'))
    thread.start()
    
    return jsonify({
        "status": "Scrape notebook execution started",
        "notebook": notebook_path,
        "message": "Please check results: https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit?gid=2038294867#gid=2038294867"
    }), 200

@app.route('/progress/run_rss', methods=['GET'])
def get_rss_progress():
    token = request.args.get('token')
    if token != 'your_secret_token':
        return jsonify({"error": "Invalid token"}), 403

    return jsonify({"step": "run_rss", "progress": progress['run_rss']}), 200

@app.route('/progress/run_scrape', methods=['GET'])
def get_scrape_progress():
    token = request.args.get('token')
    if token != 'your_secret_token':
        return jsonify({"error": "Invalid token"}), 403

    return jsonify({"step": "run_scrape", "progress": progress['run_scrape']}), 200

if __name__ == '__main__':
    app.run(host='45.133.178.134', port=35474)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://45.133.178.134:35474
Press CTRL+C to quit
185.155.16.157 - - [13/Jun/2024 15:38:32] "GET /run_rss?token=your_secret_token HTTP/1.1" 200 -
185.155.16.157 - - [13/Jun/2024 15:38:39] "GET /progress/run_rss?token=your_secret_token HTTP/1.1" 200 -
185.155.16.157 - - [13/Jun/2024 15:38:40] "GET /progress/run_rss?token=your_secret_token HTTP/1.1" 200 -
185.155.16.157 - - [13/Jun/2024 17:07:11] "GET /progress/run_rss?token=your_secret_token HTTP/1.1" 200 -
185.155.16.157 - - [13/Jun/2024 17:07:16] "GET /progress/run_scrape?token=your_secret_token HTTP/1.1" 200 -
185.155.16.157 - - [13/Jun/2024 17:07:39] "GET /progress/run_scrape?token=your_secret_token HTTP/1.1" 200 -
185.155.16.157 - - [13/Jun/2024 17:07:46] "GET /progress/run_rss?token=your_secret_token HTTP/1.1" 200 -
185.155.16.157 - - [13/Jun/2024 17:08:16] "GET /progress/run_scrape?token=your_secret_token HTTP/1.1" 200 -
185.155.16.157 - - [13/Jun/2024 17:08:19] "GET /progress/run_scrape?token=your_secret_token HTTP/

In [ ]:
execute_notebook('rss.ipynb')

In [ ]:
%run rss.ipynb